In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

## Load data

In [2]:
x_train = pd.read_csv('final_x_train.csv')
y_train = pd.read_csv('final_y_train.csv')
x_valid = pd.read_csv('final_x_valid.csv')
y_valid = pd.read_csv('final_y_valid.csv')
x_test = pd.read_csv('final_x_test.csv')
y_test = pd.read_csv('final_y_test.csv')

## Set up

In [3]:
#One hot encoding to target variable for NN
def O_H_E_Y(dataset):
    Y_data = pd.get_dummies(dataset['risk_rating'],dtype =int)
    new_column_names = ['risk_rating_' + str(i + 1) for i in range(len(Y_data.columns))]
    Y_data_renamed = Y_data.rename(columns=dict(zip(Y_data.columns, new_column_names)))
    data_combined = Y_data_renamed.join(dataset)
    Data_completed = data_combined.rename(columns = {'risk_rating_16':'risk_rating_17'})
    return Data_completed.drop(['risk_rating'],axis =1)

y_train= O_H_E_Y(y_train)
y_valid= O_H_E_Y(y_valid)
y_test= O_H_E_Y(y_test)

In [4]:
x_test.shape

(1516, 110)

In [5]:
y_valid.shape

(3029, 16)

## Model Fiiting

To enhance accuracy, the model's performance was enhanced through two main approaches: optimizing hyperparameters and adjusting the number of hidden layers. 

The hyperparameters under consideration for testing encompass hidden layer size, batch size, max epochs, optimizer, and activation method within the hidden layers. 

Regarding the number of hidden layers, evaluations were conducted individually for configurations with one to five hidden layers. 

### One Hidden Layer

In [6]:
#Set up the model with one hidden layer
def NN(hidden_layer_size=100,activation = 'tanh',optimizer='adam'):
    input_size = 110
    output_size = 16
    model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(input_size,)),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(output_size, activation='softmax')
        ])
    model.compile(optimizer=optimizer, loss='CategoricalCrossentropy', metrics=['accuracy'])
    return model 

model = KerasClassifier(model=NN,
                        hidden_layer_size=100,
                        activation = 'tanh',optimizer='adam', verbose=0)

In [7]:
# Seting up the activation methods, optimizser, number of hidden layer size, batch size, and epochs 
# to find the best combination
# Each hyperparameters will be selected one value to search the highest accuracy score. 
param_grid = {
    'hidden_layer_size' : [50,100,200],
    'activation' : ['tanh','softmax','relu','sigmoid'],
    'optimizer' : ['SGD','RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
    'batch_size' : [50,100,150],
    'epochs' : [10, 20,50]
}

Hyper_param_search = GridSearchCV(estimator=model,
                       param_grid=param_grid, scoring = 'accuracy',n_jobs=-1, cv=3, verbose=2,)

#Fitting the trainning and validation dataset
Hyper_param_search.fit(x_train, y_train, validation_data=(x_valid, y_valid))

# check the best hyperparameter
Hyper_param_search.best_estimator_, Hyper_param_search.best_params_, Hyper_param_search.best_score_

Fitting 3 folds for each of 756 candidates, totalling 2268 fits


(KerasClassifier(
 	model=<function NN at 0x000001E7420B05E0>
 	build_fn=None
 	warm_start=False
 	random_state=None
 	optimizer=Adadelta
 	loss=None
 	metrics=None
 	batch_size=50
 	validation_batch_size=None
 	verbose=0
 	callbacks=None
 	validation_split=0.0
 	shuffle=True
 	run_eagerly=False
 	epochs=10
 	hidden_layer_size=50
 	activation=tanh
 	class_weight=None
 ),
 {'activation': 'tanh',
  'batch_size': 50,
  'epochs': 10,
  'hidden_layer_size': 50,
  'optimizer': 'Adadelta'},
 0.23796944133600073)

In [8]:
# Using test set to evaluate the accuracy
best_model = Hyper_param_search.best_estimator_
test_accuracy = best_model.score(x_test, y_test)
test_accuracy

0.20712401055408972

### Two Hidden Layer

In [9]:
#Set up the model with two hidden layer
def NN2(hidden_layer_size=100,activation = 'tanh',optimizer='adam'):
    input_size = 110
    output_size = 16
    model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(input_size,)),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(output_size, activation='softmax')
        ])
    model.compile(optimizer=optimizer, loss='CategoricalCrossentropy', metrics=['accuracy'])
    return model 

model = KerasClassifier(model=NN2,
                        hidden_layer_size=100,
                        activation = 'tanh',optimizer='adam', verbose=0)

In [10]:
# Same stituation with previous coding
# Seting up the activation methods, optimizser, number of hidden layer size, batch size, and epochs 
# to find the best combination
# Each hyperparameters will be selected one value to search the highest accuracy score. 
param_grid = {
    'hidden_layer_size' : [50,100,200],
    'activation' : ['tanh','softmax','relu','sigmoid'],
    'optimizer' : ['SGD','RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
    'batch_size' : [50,100,150],
    'epochs' : [10, 20,50]

}

Hyper_param_search = GridSearchCV(estimator=model,
                       param_grid=param_grid, scoring = 'accuracy',n_jobs=-1, cv=3, verbose=2,)
Hyper_param_search.fit(x_train, y_train, validation_data=(x_valid, y_valid))

# check the best hyperparameter
Hyper_param_search.best_estimator_, Hyper_param_search.best_params_, Hyper_param_search.best_score_           

Fitting 3 folds for each of 756 candidates, totalling 2268 fits


(KerasClassifier(
 	model=<function NN2 at 0x000001E74378CF70>
 	build_fn=None
 	warm_start=False
 	random_state=None
 	optimizer=SGD
 	loss=None
 	metrics=None
 	batch_size=100
 	validation_batch_size=None
 	verbose=0
 	callbacks=None
 	validation_split=0.0
 	shuffle=True
 	run_eagerly=False
 	epochs=50
 	hidden_layer_size=50
 	activation=sigmoid
 	class_weight=None
 ),
 {'activation': 'sigmoid',
  'batch_size': 100,
  'epochs': 50,
  'hidden_layer_size': 50,
  'optimizer': 'SGD'},
 0.2324022373528903)

In [12]:
best_model = Hyper_param_search.best_estimator_
test_accuracy = best_model.score(x_test, y_test)
test_accuracy

0.2183377308707124

## Three Hidden Layer

In [4]:
#Set up the model with three hidden layer
def NN3(hidden_layer_size=100,activation = 'tanh',optimizer='adam'):
    input_size = 110
    output_size = 16
    model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(input_size,)),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(output_size, activation='softmax')
        ])
    model.compile(optimizer=optimizer, loss='CategoricalCrossentropy', metrics=['accuracy'])
    return model 

model = KerasClassifier(model=NN3,
                        hidden_layer_size=100,
                        activation = 'tanh',optimizer='adam', verbose=0)

In [5]:
# Same stituation with previous coding
# Seting up the activation methods, optimizser, number of hidden layer size, batch size, and epochs 
# to find the best combination
# Each hyperparameters will be selected one value to search the highest accuracy score. 
param_grid = {
    'hidden_layer_size' : [50,100,200],
    'activation' : ['tanh','softmax','relu','sigmoid'],
    'optimizer' : ['SGD','RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
    'batch_size' : [50,100,150],
    'epochs' : [10, 20,50]

}

Hyper_param_search = GridSearchCV(estimator=model,
                       param_grid=param_grid, scoring = 'accuracy',n_jobs=-1, cv=3, verbose=2,)

#Fitting the trainning and validation dataset
Hyper_param_search.fit(x_train, y_train, validation_data=(x_valid, y_valid))

# check the best hyperparameter
Hyper_param_search.best_estimator_, Hyper_param_search.best_params_, Hyper_param_search.best_score_              

Fitting 3 folds for each of 756 candidates, totalling 2268 fits


(KerasClassifier(
 	model=<function NN3 at 0x00000223809EE940>
 	build_fn=None
 	warm_start=False
 	random_state=None
 	optimizer=Adagrad
 	loss=None
 	metrics=None
 	batch_size=150
 	validation_batch_size=None
 	verbose=0
 	callbacks=None
 	validation_split=0.0
 	shuffle=True
 	run_eagerly=False
 	epochs=10
 	hidden_layer_size=50
 	activation=tanh
 	class_weight=None
 ),
 {'activation': 'tanh',
  'batch_size': 150,
  'epochs': 10,
  'hidden_layer_size': 50,
  'optimizer': 'Adagrad'},
 0.23334612327473359)

In [6]:
# Using test set to evaluate the accuracy
best_model = Hyper_param_search.best_estimator_
test_accuracy = best_model.score(x_test, y_test)
test_accuracy

0.22031662269129287

## Four Hidden Layer

In [7]:
#Set up the model with four hidden layer
def NN4(hidden_layer_size=100,activation = 'tanh',optimizer='adam'):
    input_size = 110
    output_size = 16
    model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(input_size,)),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(output_size, activation='softmax')
        ])
    model.compile(optimizer=optimizer, loss='CategoricalCrossentropy', metrics=['accuracy'])
    return model 

model = KerasClassifier(model=NN4,
                        hidden_layer_size=100,
                        activation = 'tanh',optimizer='adam', verbose=0)

In [8]:
# Same stituation with previous coding
# Seting up the activation methods, optimizser, number of hidden layer size, batch size, and epochs 
# to find the best combination
# Each hyperparameters will be selected one value to search the highest accuracy score. 
param_grid = {
    'hidden_layer_size' : [50,100,200],
    'activation' : ['tanh','softmax','relu','sigmoid'],
    'optimizer' : ['SGD','RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
    'batch_size' : [50,100,150],
    'epochs' : [10, 20,50]

}

Hyper_param_search = GridSearchCV(estimator=model,
                       param_grid=param_grid, scoring = 'accuracy',n_jobs=-1, cv=3, verbose=2,)

#Fitting the trainning and validation dataset
Hyper_param_search.fit(x_train, y_train, validation_data=(x_valid, y_valid))

# check the best hyperparameter
Hyper_param_search.best_estimator_, Hyper_param_search.best_params_, Hyper_param_search.best_score_ 

Fitting 3 folds for each of 756 candidates, totalling 2268 fits


(KerasClassifier(
 	model=<function NN4 at 0x00000223EC8F6310>
 	build_fn=None
 	warm_start=False
 	random_state=None
 	optimizer=SGD
 	loss=None
 	metrics=None
 	batch_size=100
 	validation_batch_size=None
 	verbose=0
 	callbacks=None
 	validation_split=0.0
 	shuffle=True
 	run_eagerly=False
 	epochs=50
 	hidden_layer_size=200
 	activation=sigmoid
 	class_weight=None
 ),
 {'activation': 'sigmoid',
  'batch_size': 100,
  'epochs': 50,
  'hidden_layer_size': 200,
  'optimizer': 'SGD'},
 0.22947697901370426)

In [9]:
# Using test set to evaluate the accuracy
best_model = Hyper_param_search.best_estimator_
test_accuracy = best_model.score(x_test, y_test)
test_accuracy

0.21108179419525067

## Five Hidden Layer

In [10]:
#Set up the model with five hidden layer
def NN5(hidden_layer_size=100,activation = 'tanh',optimizer='adam'):
    input_size = 110
    output_size = 16
    model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(input_size,)),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(hidden_layer_size, activation=activation),
            tf.keras.layers.Dense(output_size, activation='softmax')
        ])
    model.compile(optimizer=optimizer, loss='CategoricalCrossentropy', metrics=['accuracy'])
    return model 

model = KerasClassifier(model=NN5,
                        hidden_layer_size=100,
                        activation = 'tanh',optimizer='adam', verbose=0)

In [11]:
# Same stituation with previous coding
# Seting up the activation methods, optimizser, number of hidden layer size, batch size, and epochs 
# to find the best combination
# Each hyperparameters will be selected one value to search the highest accuracy score. 
param_grid = {
    'hidden_layer_size' : [50,100,200],
    'activation' : ['tanh','softmax','relu','sigmoid'],
    'optimizer' : ['SGD','RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
    'batch_size' : [50,100,150],
    'epochs' : [10, 20,50]

}

Hyper_param_search = GridSearchCV(estimator=model,
                       param_grid=param_grid, scoring = 'accuracy',n_jobs=-1, cv=3, verbose=2,)

#Fitting the trainning and validation dataset
Hyper_param_search.fit(x_train, y_train, validation_data=(x_valid, y_valid))

# check the best hyperparameter
Hyper_param_search.best_estimator_, Hyper_param_search.best_params_, Hyper_param_search.best_score_  

Fitting 3 folds for each of 756 candidates, totalling 2268 fits


(KerasClassifier(
 	model=<function NN5 at 0x00000223EA880280>
 	build_fn=None
 	warm_start=False
 	random_state=None
 	optimizer=Adadelta
 	loss=None
 	metrics=None
 	batch_size=100
 	validation_batch_size=None
 	verbose=0
 	callbacks=None
 	validation_split=0.0
 	shuffle=True
 	run_eagerly=False
 	epochs=50
 	hidden_layer_size=50
 	activation=sigmoid
 	class_weight=None
 ),
 {'activation': 'sigmoid',
  'batch_size': 100,
  'epochs': 50,
  'hidden_layer_size': 50,
  'optimizer': 'Adadelta'},
 0.2302325151510586)

In [12]:
# Using test set to evaluate the accuracy
best_model = Hyper_param_search.best_estimator_
test_accuracy = best_model.score(x_test, y_test)
test_accuracy

0.20316622691292877

### Conclusion 

In summary, the neural network exhibited its highest performance, achieving an accuracy rate of 22.03% with a single hidden layer and the sigmoid activation function.